In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Master Template for processing each dataset
- Sentinel-2 (in order of priority)
 - **L2A via GCloud**: in testing
 - **L1C via GCloud processed to L2A**: under development
 - L1C via ESA processed to L2A: partially tested
 - L2A via ESA: partially tested
 - L1C via GCloud: partially tested
 - L1C via ESA: partiall tested

In [12]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time
import tqdm
import os

from subprocess import Popen, PIPE, STDOUT

from utils.prepS2 import prepareS2
os.environ['AWS_ACCESS']=""
os.environ['AWS_SECRET']=""


In [13]:
des_scenes_df = pd.read_csv('./s2l2agcloudhitlist.csv')
des_scenes_list = list(des_scenes_df.PRODUCT_ID.values)

In [5]:
des_scenes_list

['S2A_MSIL2A_20181213T231841_N0211_R001_T58LEN_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LEP_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LFN_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LFP_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LGN_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LGP_20181214T005852',
 'S2A_MSIL2A_20181213T231841_N0211_R001_T58LGQ_20181214T005852',
 'S2B_MSIL2A_20181214T000729_N0211_R073_T56MQS_20181214T014149',
 'S2B_MSIL2A_20181214T000729_N0211_R073_T56MQT_20181214T014149',
 'S2B_MSIL2A_20181214T000729_N0211_R073_T56MRT_20181214T014149',
 'S2B_MSIL2A_20181214T233809_N0211_R087_T57LWH_20181215T012405',
 'S2B_MSIL2A_20181214T233809_N0211_R087_T57LWK_20181215T012405',
 'S2B_MSIL2A_20181214T233809_N0211_R087_T57LXG_20181215T012405',
 'S2B_MSIL2A_20181214T233809_N0211_R087_T57LXH_20181215T012405',
 'S2B_MSIL2A_20181214T233809_N0211_R087_T57LXK_20181215T012405',
 'S2B_MSIL2A_20181214T233

In [14]:
os.environ['COPERNICUS_USERNAME']="..."
os.environ['COPERNICUS_PWD']="..."

os.environ['AWS_ACCESS']=""
os.environ['AWS_SECRET']=""

In [15]:
out_dir = '../S2_ARD/'

# output directory currently not cloud bucket, it's just used as ref for intermediate products.
# these test scenes are hardcoded to be uploaded to: public-eo-data/fiji/Sentinel_2_test/
for scene_nm in tqdm.tqdm(des_scenes_list):
     prepareS2(scene_nm, out_dir=['public-eo-data', 'fiji/Sentinel_2_testmulti/'], inter_dir='../S2_ARD/')

  0%|          | 0/2786 [00:00<?, ?it/s]

Will upload to S3 bucket: ['public-eo-data', 'fiji/Sentinel_2_testmulti/']
[('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEN_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEN_20181214T005852.SAFE/GRANULE/L2A_T58LEN_A018159_20181213T231844/IMG_DATA/R10m/T58LEN_20181213T231841_AOT_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEN_tmp/S2A_MSIL2A_20181213T231841_T58LEN/S2A_MSIL2A_20181213T231841_T58LEN_AOT_10m.tif'), ('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEN_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEN_20181214T005852.SAFE/GRANULE/L2A_T58LEN_A018159_20181213T231844/IMG_DATA/R10m/T58LEN_20181213T231841_B02_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEN_tmp/S2A_MSIL2A_20181213T231841_T58LEN/S2A_MSIL2A_20181213T231841_T58LEN_B02_10m.tif'), ('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEN_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEN_20181214T005852.SAFE/GRANULE/L2A_T58LEN_A018159_20181213T231844/IMG_DATA/R10m/T58LEN_20181213T231841_B03_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213

  0%|          | 1/2786 [04:49<223:42:19, 289.17s/it]

Will upload to S3 bucket: ['public-eo-data', 'fiji/Sentinel_2_testmulti/']
[('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEP_20181214T005852.SAFE/GRANULE/L2A_T58LEP_A018159_20181213T231844/IMG_DATA/R10m/T58LEP_20181213T231841_AOT_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_AOT_10m.tif'), ('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEP_20181214T005852.SAFE/GRANULE/L2A_T58LEP_A018159_20181213T231844/IMG_DATA/R10m/T58LEP_20181213T231841_B02_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B02_10m.tif'), ('../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LEP_20181214T005852.SAFE/GRANULE/L2A_T58LEP_A018159_20181213T231844/IMG_DATA/R10m/T58LEP_20181213T231841_B03_10m.jp2', '../S2_ARD/S2A_MSIL2A_20181213

Process ForkPoolWorker-14:
Process ForkPoolWorker-18:
Process ForkPoolWorker-12:
Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-20:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-16:
Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-17:
Process ForkPoolWorker-19:
Process ForkPoolWorker-2:
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/tj/miniconda

...

Local source file: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B12_20m.tif


  File "/home/tj/miniconda3/envs/cs/lib/python3.7/site-packages/boto3/session.py", line 263, in client
    aws_session_token=aws_session_token, config=config)


S3 target file: fiji/Sentinel_2_testmulti/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B12_20m.tif


  File "/home/tj/IPP_Common_Sensing/IPP_Common_Sensing/utils/prepS2.py", line 328, in s3_single_upload
    's3',
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/site-packages/botocore/session.py", line 839, in create_client
    client_config=config, api_version=api_version)


Start: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B12_20m.tif 2019-09-03 15:28:09 
Local source file: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B8A_20m.tif
S3 target file: fiji/Sentinel_2_testmulti/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B8A_20m.tif
Start: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_B8A_20m.tif 2019-09-03 15:28:09 
Local source file: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_SCL_20m.tif
S3 target file: fiji/Sentinel_2_testmulti/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_SCL_20m.tif
Start: ../S2_ARD/S2A_MSIL2A_20181213T231841_T58LEP_tmp/S2A_MSIL2A_20181213T231841_T58LEP/S2A_MSIL2A_20181213T231841_T58LEP_SCL_20m.tif 2019-09-03 15:2

## Legacy Example... will not work, just for reference...

In [3]:
non_cogs_dir = "../S2_Download/"
cogs_dir = "../S2_ARD/"
log_file = "./processLog.csv"
sen2cor_8 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"
sen2cor_5 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"

s3_bucket = "public-eo-data"
s3_dir = "catapult/sentinel2/"
aws_creds = pd.read_csv("../aws_creds.csv")
access_key = aws_creds.AWSAccessKeyId.values[0]
secret_key = aws_creds.AWSSecretKey.values[0]

In [4]:
des_scenes_list = [
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWF_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KXE_20180821T013410.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXE_20190608T233246.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXF_20190608T233246.SAFE",
    "S2A_MSIL1C_20160930T233812_N0204_R087_T57LWK_20160930T233815.SAFE",
    "S2A_MSIL1C_20160930T233812_N0204_R087_T57LXK_20160930T233815.SAFE",
    "S2B_MSIL1C_20171222T234729_N0206_R130_T57LWK_20171223T005857.SAFE",
    "S2B_MSIL1C_20180725T230019_N0206_R058_T58KHF_20180726T002918.SAFE"
    
]

In [8]:
[find_s2_uuid(x) for x in des_scenes_list]

['da9064f8-5186-408c-8da5-3347d4ae5b76',
 '77da6be6-ddfb-4a64-ad58-085c07a6fb7f',
 '56e18fe8-d091-4540-bcff-5e1bb4f7581a',
 '132890a6-1807-49d7-a7b0-48a3331d0c8d',
 'f95fb5bd-ae2a-4285-aa0c-066a8b92a5f4',
 '39ab4e1d-a52d-47dd-9a98-2c3fa230d0f4',
 '8ae5481e-b9b8-44fb-9cd3-2d09520a4dd9',
 '52713300-23ec-4d6d-af8b-92c1fd7bab87',
 '7b837bd6-4cca-4312-88cc-532fa4b8af76']

In [ ]:
with open(log_file, 'a') as log:

    for des_scene in des_scenes_list:
        log.write("{},{},{}".format(des_scene, 'Starting', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        # shorten scene name
        scene_name = des_scene[:-21]
        scene_name = scene_name[:-17] + scene_name.split('_')[-1]
        print ( 'Scene name: {}'.format(scene_name) )

        # find uuid for download via esa hub
        s2id = find_s2_uuid(des_scene)

        down_dir = non_cogs_dir + des_scene + '/'
        print ( 'Download scene dir: {}'.format(down_dir) )
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
        download_s2_granule_gcloud(des_scene, non_cogs_dir)
#         download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence

        log.write("{},{},{}".format(des_scene, 'Downloaded', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # process to l2a
#         l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
#         print ( 'L2A temp dir: {}'.format(l2a_dir) )
#         if not os.path.exists(l2a_dir):
#             sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...

        log.write("{},{},{}".format(des_scene, 'Corrected', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # convert to cog
        cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
        print ( 'COG dir: {}'.format(cog_dir) )
        conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        log.write("{},{},{}".format(des_scene, 'COGGED', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        create_yaml(cog_dir, 's2')

        log.write("{},{},{}".format(des_scene, 'Yamled', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )



Scene name: S2A_MSIL1C_20180820T223011_T60KWE
Download scene dir: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
ESA scene already extracted: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
L2A temp dir: ../S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
COG dir: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_AOT_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B02_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B03_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B04_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2

Downloading:   3%|▎         | 23.1M/744M [00:25<11:50, 1.01MB/s]

In [ ]:
with open(log_file, 'a') as log:

    for des_scene in des_scenes_list:
        log.write("{},{},{}".format(des_scene, 'Starting', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        # shorten scene name
        scene_name = des_scene[:-21]
        scene_name = scene_name[:-17] + scene_name.split('_')[-1]
        print ( 'Scene name: {}'.format(scene_name) )

        # find uuid for download via esa hub
        s2id = find_s2_uuid(des_scene)

        down_dir = non_cogs_dir + des_scene + '/'
        print ( 'Download scene dir: {}'.format(down_dir) )
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
        #download_s2_granule_gcloud(des_scene, non_cogs_dir)
        download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence

        log.write("{},{},{}".format(des_scene, 'Downloaded', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # process to l2a
        l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
        print ( 'L2A temp dir: {}'.format(l2a_dir) )
        if not os.path.exists(l2a_dir):
            sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...

        log.write("{},{},{}".format(des_scene, 'Corrected', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # convert to cog
        cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
        print ( 'COG dir: {}'.format(cog_dir) )
        conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        log.write("{},{},{}".format(des_scene, 'COGGED', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        create_yaml(cog_dir, 's2')

        log.write("{},{},{}".format(des_scene, 'Yamled', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

